In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from googletrans import Translator
from tqdm import tqdm
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk import tokenize
import nltk
import seaborn as sns
import unidecode
from string import punctuation
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

In [ ]:
# ds_columns  = ["sentiment", "ids", "date", "flag", "user", "text"]
ds_encoding = "ISO-8859-1"
df_tweet = pd.read_csv(r'D:\Estudos\FIAP\Fase 5\Trabalhos_FIAP\arquivo_traduzido.csv', sep=',')
df_tweet.head()

In [ ]:
df_tweet.info()

In [ ]:
vetorizar = CountVectorizer(lowercase=False, max_features=50)
bag_of_words = vetorizar.fit_transform(df_tweet.text_pt)
# print(bag_of_words)
# matriz_esparsa = pd.DataFrame.sparse.from_spmatrix(bag_of_words, columns=vetorizar.get_feature_names())
# print(matriz_esparsa)

In [ ]:
def classificacao_texto(df, coluna_texto, coluna_classificacao):
    vetorizar = CountVectorizer(lowercase=False, max_features=2000)
    bag_of_words = vetorizar.fit_transform(df[coluna_texto])
    # matriz_esparsa = pd.DataFrame.sparse.from_spmatrix(bag_of_words, columns=vetorizar.get_feature_names())
    
    treino, teste, classe_treino, classe_teste = train_test_split(bag_of_words, 
                                                                    df[coluna_classificacao], 
                                                                    random_state=42)
    regressao_logistica = LogisticRegression()
    regressao_logistica.fit(treino,classe_treino)
    return regressao_logistica.score(teste, classe_teste)


In [ ]:
vectorizer = CountVectorizer(analyzer = "word")
freq_tweets = vectorizer.fit_transform(df_tweet.text_pt)

modelo = MultinomialNB()
modelo.fit(freq_tweets, df_tweet.sentiment)

In [ ]:
testes = ["Esse governo está no início, vamos ver o que vai dar",
          "Estou muito feliz com o governo de São Paulo esse ano",
          "O estado de Minas Gerais decretou calamidade financeira!!!",
          "A segurança desse país está deixando a desejar",
          "O governador de Minas é do PT",
          "O prefeito de São Paulo está fazendo um ótimo trabalho"]

freq_testes = vectorizer.transform(testes)
modelo.predict(freq_testes)

In [ ]:
resultados = cross_val_predict(modelo, freq_tweets, df_tweet.sentiment, cv = 10)
resultados

In [ ]:
metrics.accuracy_score(df_tweet.sentiment, resultados)

In [ ]:
def pareto(texto, coluna_texto, quantidade):
  todas_palavras = ' '.join([resenha for resenha in texto[coluna_texto]])
  token_espaco = nltk.tokenize.WhitespaceTokenizer()
  token_frase = token_espaco.tokenize(todas_palavras)
  frequencias = nltk.FreqDist(token_frase)
  df_frequencias = pd.DataFrame({'Palavras': list(frequencias.keys()),
                               'Frequencia': list(frequencias.values())})
  df_frequencias = df_frequencias.nlargest(n=quantidade, columns='Frequencia')

  total = df_frequencias['Frequencia'].sum()
  df_frequencias['Porcentagem'] = df_frequencias['Frequencia'].cumsum() / total * 100

  plt.figure(figsize=(12,8))
  ax = sns.barplot(data=df_frequencias, x='Palavras', y='Frequencia', color='gray')
  ax2 = ax.twinx()
  sns.lineplot(data=df_frequencias, x='Palavras', y='Porcentagem', color='red', sort=False, ax=ax2)
  plt.show()

In [ ]:
pareto(df_tweet,'text_pt',10)

In [ ]:
def nuvem_palavras(texto, coluna_texto):
    todas_palavras = ' ' .join([texto for texto in texto[coluna_texto]])
    nuvem_palavras = WordCloud(width=400, height=250, max_font_size=50, collocations=False).generate(todas_palavras)
    plt.figure(figsize=(10,7))
    plt.imshow(nuvem_palavras,interpolation='bilinear')
    plt.show()
nuvem_palavras(df_tweet, 'text_pt')

In [ ]:
todas_palavras = ' ' .join([texto for texto in df_tweet.text_pt])
token_espaco = tokenize.WhitespaceTokenizer()
token_frase = token_espaco.tokenize(todas_palavras)
frequencia = nltk.FreqDist(token_frase)

In [ ]:
df_frequencia = pd.DataFrame({
                    'Palavra':list(frequencia.keys()),
                    'Frequencia':list(frequencia.values())})

In [ ]:
df_frequencia.nlargest(columns='Frequencia',n=10)

In [ ]:
palavras_irrelevantes = nltk.corpus.stopwords.words('portuguese')

tweet_processado = list()
for tweet in df_tweet.text_pt:
    novo_tweet = list()
    palavras_texto = token_espaco.tokenize(tweet)
    for palavra in palavras_texto:
        if palavra not in palavras_irrelevantes:
            novo_tweet.append(palavra)
    tweet_processado.append(' '.join(novo_tweet))

df_tweet['text_pt_v2'] = tweet_processado

In [ ]:
token_pontuacao = tokenize.WordPunctTokenizer()
pontuacao = list()
for ponto in punctuation:
    pontuacao.append(ponto)

pontuacao_stopwords = pontuacao + palavras_irrelevantes

tweet_processado = list()
for tweet in df_tweet['text_pt_v2']:
    novo_tweet = list()
    palavras_texto = token_pontuacao.tokenize(tweet)
    for palavra in palavras_texto:
        if palavra not in pontuacao_stopwords:
            novo_tweet.append(palavra)
    tweet_processado.append(' '.join(novo_tweet))

df_tweet['text_pt_v3'] = tweet_processado

In [ ]:
sem_acentos = [unidecode.unidecode(texto) for texto in df_tweet.text_pt_v3]
stopwords = [unidecode.unidecode(texto) for texto in pontuacao_stopwords]

In [ ]:
df_tweet['text_pt_v4'] = sem_acentos
stemmer = nltk.RSLPStemmer()

tweet_processado = list()
for tweet in df_tweet.text_pt_v4:
    novo_tweet = list()
    tweet = tweet.lower()
    palavras_texto = token_pontuacao.tokenize(tweet)
    for palavra in palavras_texto:
        if palavra not in pontuacao_stopwords:
            novo_tweet.append(stemmer.stem(palavra))
    tweet_processado.append(' '.join(novo_tweet))

df_tweet['text_pt_v4'] = tweet_processado

In [ ]:
pareto(df_tweet,'text_pt_v4',10)

In [ ]:
nuvem_palavras(df_tweet, 'text_pt_v4')

In [ ]:
palavras_irrelevantes_v2 = ['agor','aind', '..', '...', 'dia','tod','faz','tao','noit', 
                            'quer','pod', 'nov','esper','!!','cas','enta','hoj','poss',
                            'ach','apen','vai','vou','2','volt','est','ter','twitt','acab',
                            'temp','seman','assist','voc','ir','algum','!!!','dev','sab','falt','cois','trabalh']

tweet_processado = []
for tweet in df_tweet.text_pt_v4:
    novo_tweet = []
    palavras_texto = token_espaco.tokenize(tweet)
    for palavra in palavras_texto:
        if palavra not in palavras_irrelevantes_v2:
            novo_tweet.append(palavra)
    tweet_processado.append(' '.join(novo_tweet))

df_tweet['text_pt_v5'] = tweet_processado

In [ ]:
pareto(df_tweet,'text_pt_v5',20)

In [ ]:
print(f"Acuracia sem tratamento: {classificacao_texto(df_tweet, 'text_pt', 'sentiment')}")
print(f"Acuracia após o 1º tratamento: {classificacao_texto(df_tweet, 'text_pt_v2', 'sentiment')}")
print(f"Acuracia após o 2º tratamento: {classificacao_texto(df_tweet, 'text_pt_v3', 'sentiment')}")
print(f"Acuracia após o 3º tratamento: {classificacao_texto(df_tweet, 'text_pt_v4', 'sentiment')}")
print(f"Acuracia após o 4º tratamento: {classificacao_texto(df_tweet, 'text_pt_v5', 'sentiment')}")